Here's your updated blog with partner references removed and an analytics section added:

---

# Trimester Retrospective

## Overview

This trimester I split my focus across two areas: database consolidation and migration infrastructure and Spring backend security hardening. After the DB work stabilized, I went deep on security — auditing the auth system, hardening the password reset flow, locking down endpoints, and fixing cookie handling. The security work ended up being much larger in scope than I originally anticipated.

---

## Analytics

![GitHub Analytics]({{site.baseurl}}/images/githubanalytics.png)
![Grade Predictor]({{site.baseurl}}/images/gradepredictor.png)

Numerous pull requests, commits, and merges across both Spring and Flask repositories. The focus was on database reliability, schema consolidation, and security hardening.

[Spring Pull Requests](https://github.com/Open-Coding-Society/spring/pulls?q=is%3Apr+author%3AHypernova101)

[Spring Pull Requests made by Yash, but still could have my commits](https://github.com/Open-Coding-Society/spring/pulls?q=is%3Apr+author%illuminati1618)

[Flask Commits](https://github.com/Open-Coding-Society/flask/commits?author=Hypernova101)

---

## Part 1 — Database Consolidation & Migration Infrastructure

### Schema Consolidation

The RPG side of the backend had a redundancy problem. Five separate tables and controllers (`AdventureAnswer`, `AdventureChoice`, `AdventureQuestion`, `AdventureRubric`, `GameStat`) existed for what was essentially one concept. I collapsed them:

- Merged all five adventure tables into a single `Adventure` entity with a `details` JSON blob — one controller, one repository, one `/adventure/combined/{personId}` endpoint that returns everything at once
- `ModelInit` handled the live migration: added the `details` column on startup, then walked existing rows and serialized the old fields into JSON so no data was lost
- Did the same for games — unified blackjack, cryptomining, casino, and bank transactions into a single `Game` entity with a `type` field, replacing all scattered repositories with `UnifiedGameRepository`
- Removed the dead `CryptoJPArepo` entirely — the crypto controller had been hitting a stale DB table instead of the live API
- Built Thymeleaf admin read/edit views for both Adventure and Games, gated behind `ROLE_ADMIN`
- Consolidated 4 backup controllers down to 2 with proper file rotation

**Net result on adventure alone: -1,045 lines replaced by 363 that do more.**

| PR | Description |
|---|---|
| [PR #44](https://github.com/Open-Coding-Society/spring/pull/44) | Collapse 5 adventure tables → 1 unified entity |
| [PR #48](https://github.com/Open-Coding-Society/spring/pull/48) | Unified Game entity, MVC admin templates, admin-only gating, JSON formatting |

---

### Migration Infrastructure

- End-to-end migration system for Spring with flag handling to prevent race conditions during schema recreation — if two processes tried to recreate the schema simultaneously, it would fail silently. Wrapping flag cleanup in try-finally blocks was the fix.
- Fixed the Flask `db_init.py` script to handle MySQL's foreign key constraint behavior — the existing script worked on SQLite locally but would crash in production because MySQL actually enforces foreign key checks during `DROP TABLE`. The fix disables checks before dropping, then re-enables them before recreation:

```python
if db.engine.url.drivername in ['mysql', 'mysql+pymysql']:
    db.session.execute(db.text('SET FOREIGN_KEY_CHECKS=0;'))
    db.session.commit()
db.drop_all()
if db.engine.url.drivername in ['mysql', 'mysql+pymysql']:
    db.session.execute(db.text('SET FOREIGN_KEY_CHECKS=1;'))
    db.session.commit()
```

| PR | Description |
|---|---|
| [PR #38](https://github.com/Open-Coding-Society/spring/pull/38) | Database migration infrastructure |
| [PR #40](https://github.com/Open-Coding-Society/spring/pull/40) | Migration enhancements & flag handling |
| [PR #63](https://github.com/Open-Coding-Society/spring/pull/63) | DB script update for new schema |
| [Flask PR #29](https://github.com/Open-Coding-Society/flask/pull/29) | Fix db_init for MySQL foreign key constraints |

### Production Migration

Running the actual migration on AWS Cockpit was the stressful part. Multiple developers were committing concurrent schema changes during the migration window, creating conflicts I had to resolve live. A lot of that work doesn't show up as commits — it was terminal sessions and coordination — but without it things would have broken badly.

---

## Part 2 — Security Hardening

After migration, I shifted entirely to security. What started as patching one vulnerability turned into a full audit of the Spring backend's auth system, endpoint policies, and cookie handling.

---

### Chapter 1 — Path Traversal Vulnerability

#### The Discovery

While reviewing the Spring backend I found path traversal vulnerabilities in the import and upload endpoints. The issue was subtle — `Paths.get()` looks safe but doesn't prevent traversal. User-supplied filenames were being joined directly to base paths:

```java
// Vulnerable
Path filePath = Paths.get(backupBasePath, directoryName, filename);
// Accepts: ../../../../etc/passwd, /absolute/path
```

An authenticated attacker could hit `/api/imports/view?filename=../../../../etc/passwd` and read arbitrary files off the server. The upload endpoint had the same problem on the write side.

#### The Fix

```java
Path baseDir = Paths.get(backupBasePath, directoryName).toAbsolutePath().normalize();
Path filePath = baseDir.resolve(filename).normalize();
if (!filePath.startsWith(baseDir)) {
    return ResponseEntity.badRequest().body(result);
}
```

The pattern is: normalize → resolve → verify containment. Three steps, every time, on every endpoint that accepts user-supplied paths. For uploads I also extracted only the basename before doing anything with the filename, stripping any path components before they could be used.

| PR / Commit | Description |
|---|---|
| [Commit 3f2656](https://github.com/illuminati1618/spring-databaseguys-sprint4/commit/3f265628ffa6552c54d81466a61ec1bf2dac418f) | Path traversal fix |
| [PR #66](https://github.com/Open-Coding-Society/spring/pull/66) | CodeRunner security |

---

### Chapter 2 — Cookie & Auth Stabilization

#### The Problem

The backend had a split-brain auth situation: JWT cookies and session cookies coexisted without a unified logout model. The JWT cookie was also `httpOnly(false)`, meaning JavaScript could read it — a basic XSS vulnerability. Logout only cleared the session cookie, leaving the JWT alive. The filter logic was also routing based on an `X-Origin: client` header, which any attacker could spoof.

#### The Fix

- Set JWT cookie to `httpOnly(true)` and scoped it to `/api` instead of `/`
- Built environment-aware `secureFlag` logic — `Lax` + HTTP for local dev, `None; Secure` for production — so the cookie config actually works in both environments without manual changes
- Unified logout to explicitly expire both `jwt_java_spring` and `sess_java_spring` with `maxAge(0)`
- Replaced the spoofable `X-Origin` header check with URI-based routing (`requestUri.startsWith("/api/")`)
- Added login success handler that mints and sets the JWT cookie immediately on MVC form login, so the frontend has a valid token right after logging in
- Added `resolveUid()` so users can authenticate with either their UID or email
- Wrote `api_smoke_test.py` — a standalone script to verify auth, cookie behavior, and endpoint access without needing a browser

| PR | Description |
|---|---|
| [PR #91](https://github.com/Open-Coding-Society/spring/pull/91) | Cookie fixes, unified logout, JWT hardening, smoke test |
| [PR #86](https://github.com/Open-Coding-Society/spring/pull/86) | Migration file size fix + multipart upload config |

---

### Chapter 3 — Profile Security & Audit Trails

#### The Problem

The profile update endpoints had no ownership enforcement. Any authenticated user could technically update another user's profile by hitting the API directly with a different target ID. There was also no audit logging anywhere — if something went wrong, there was no way to trace who changed what.

#### The Fix

- Added cross-user update blocking: non-admins can only update their own profile, enforced in both the MVC controller and API controller
- Blocked non-admin UID changes entirely — UID is effectively an identity anchor and shouldn't be mutable by regular users
- Required `currentPassword` verification before allowing email or password changes for non-admins
- Added `AUDIT` log entries on every sensitive operation: profile updates, blocked attempts, role changes, reset token events — all tagged with actor, target, fields changed, and whether the actor was an admin
- Added the same ownership and current-password checks to the PUT endpoint (`/api/person/{id}`) which previously had none of these guards

| PR | Description |
|---|---|
| [PR #93](https://github.com/Open-Coding-Society/spring/pull/93) | Profile security & audit trails |

---

### Chapter 4 — Password Reset & Endpoint Hardening

#### The Problem

The password reset system stored codes in a plain `ArrayList<String[]>` — no expiration enforcement, no rate limiting, no cryptographic integrity. A user could request unlimited reset codes. The endpoint policy also had a `permitAll()` fallback that let anything through by default, and a number of endpoints that should have required auth were wide open.

#### The Fix

**Reset code overhaul:**
- Replaced the ArrayList with HMAC-SHA256 signed tokens stored in a `ConcurrentHashMap`
- Tokens are single-use — `validateAndConsume()` removes the token after one successful validation
- Added a rolling-window rate limiter: max 3 requests per 15-minute window per UID
- Added `AUDIT` logging for every token event: issued, blocked, consumed, expired, failed validation
- `RESET_TOKEN_SECRET` loaded from environment variable, with a warning logged if it falls back to an ephemeral in-memory key

**Endpoint policy:**
- Replaced the `permitAll()` fallback with `anyRequest().authenticated()`
- Locked down previously open endpoints: jokes, leaderboard, gamer scores, analytics, AI preferences, chat — all now require at least `ROLE_USER`
- Added `ROLE_USER` to the UID lookup endpoint so basic authenticated users can access it, not just students/teachers/admins
- Added explicit HTTP method-level policies for MVC routes (GET vs POST on update, reset, delete)
- Documented endpoint-to-role mapping as a `@Bean` for easy auditing

| PR | Description |
|---|---|
| [PR #94](https://github.com/Open-Coding-Society/spring/pull/94) | Password reset overhaul & endpoint role hardening |

---

## Reflection

The biggest shift in my thinking this trimester happened during the path traversal work. That vulnerability wasn't obvious — `Paths.get()` is standard Java and looks completely normal in a code review. It took reading an external writeup on a similar Flask vulnerability and then carefully tracing our own code before the attack surface became clear. That taught me that security issues hide in plain sight behind familiar APIs, and that reading about vulnerabilities in other codebases is genuinely useful for finding them in your own.

The auth work reinforced something different: security debt compounds. The JWT cookie being `httpOnly(false)`, the spoofable `X-Origin` header, the logout not clearing both cookies — none of these were individually catastrophic, but together they created a system where a motivated attacker had multiple angles. Each fix felt small in isolation but the cumulative effect was a meaningfully hardened auth system.

The audit logging work changed how I think about accountability in software. Before adding those log lines I realized there was no way to answer basic questions like "who changed this person's email?" or "how many reset codes has this user requested?" That's not acceptable for a platform handling student data. The `AUDIT` prefix on every sensitive log event was a deliberate choice — it makes those entries greppable and separable from noise.

Next sprint: continue working through the Spring security fix plan — secrets governance, getting secrets out of `application.properties`, and role taxonomy normalization.